In [ ]:
!pip install git+https://github.com/huggingface/peft
!pip install transformers==4.40.1
!pip install peft==0.10.0
!pip install accelerate einops datasets bitsandbytes
from kaggle_secrets import UserSecretsClient
import os
# user_secrets = UserSecretsClient()
# githubkey = user_secrets.get_secret("GITHUB")

!rm -rf AIC2024_Phi_version
!git clone https://github.com/phuvinhnguyen/AIC2024_Phi_version.git
import sys
sys.path.append("AIC2024_Phi_version")
!pip list

In [2]:
import torch
from dataset.dataset import AICDataset, CombineAICDataset
from dataset.collator import VidLangCollator
from transformers import AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from model.llamaer_configuration import *
from model.modeling_llamaer import *
from peft import LoraConfig, get_peft_model

def count_parameters(model):
    print(f'trainable_params: {sum(p.numel() for p in model.parameters() if p.requires_grad)}')
    print(f'total_params: {sum(p.numel() for p in model.parameters())}')

model_name = 'meta-llama/Llama-3.2-1B'
TRAIN_CAPTIONS_DIR = '/kaggle/input/aic24-glob/wts_dataset_zip/annotations/caption/train'
TRAIN_CAPTIONS_TRIM_DIR = '/kaggle/input/aic24-glob/wts_dataset_zip/annotations/caption/train/normal_trimmed'
TRAIN_CAPTIONS_EXTERNAL_DIR = '/kaggle/input/aic24trainingexternal/external/BDD_PC_5K/annotations/caption/train'
EVAL_CAPTIONS_DIR = '/kaggle/input/aic24-glob/wts_dataset_zip/annotations/caption/val'
EVAL_CAPTIONS_TRIM_DIR = '/kaggle/input/aic24-glob/wts_dataset_zip/annotations/caption/val/normal_trimmed'
EVAL_CAPTIONS_EXTERNAL_DIR = '/kaggle/input/aic24trainingexternal/external/BDD_PC_5K/annotations/caption/val'

TRAIN_VIDEOS_DIR = '/kaggle/input/aic24-videos-features-every-second/matrix/train'
EVAL_VIDEOS_DIR = '/kaggle/input/aic24-videos-features-every-second/matrix/val'
TRAIN_VIDEOS_EXTERNAL_DIR = '/kaggle/input/aic24videxternal/matrix/trainExternal'
EVAL_VIDEOS_EXTERNAL_DIR = '/kaggle/input/aic24videxternal/matrix/evalExternal'

# Use DoRA for finetune llm
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'dense'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

config = LlamaerConfig()
model = VideoLlamaForCausalLM(config)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.add_adapter(lora_config)

## Use this to load weight from checkpoint (torch checkpoint)
# model.load_state_dict(torch.load('/kaggle/input/phiver-checkpoints/original'))

2024-03-06 17:34:23.221341: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-06 17:34:23.221488: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-06 17:34:23.377258: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

<All keys matched successfully>

In [3]:
model.double_perceiver.requires_grad_(True)
model.embeding_condition.requires_grad_(True)
count_parameters(model)

## Create dataset
train_dataset_normal = AICDataset(tokenizer=tokenizer, captions_path=TRAIN_CAPTIONS_DIR, videos_path=TRAIN_VIDEOS_DIR)
train_dataset_trim = AICDataset(tokenizer=tokenizer, captions_path=TRAIN_CAPTIONS_TRIM_DIR, videos_path=TRAIN_VIDEOS_DIR)
train_dataset_external = AICDataset(tokenizer=tokenizer, captions_path=TRAIN_CAPTIONS_EXTERNAL_DIR, videos_path=TRAIN_VIDEOS_EXTERNAL_DIR, dataset_type=1)

eval_dataset_normal = AICDataset(tokenizer=tokenizer, captions_path=EVAL_CAPTIONS_DIR, videos_path=EVAL_VIDEOS_DIR)
eval_dataset_trim = AICDataset(tokenizer=tokenizer, captions_path=EVAL_CAPTIONS_TRIM_DIR, videos_path=EVAL_VIDEOS_DIR)
eval_dataset_external = AICDataset(tokenizer=tokenizer, captions_path=EVAL_CAPTIONS_EXTERNAL_DIR, videos_path=EVAL_VIDEOS_EXTERNAL_DIR, dataset_type=1)

train_dataset = CombineAICDataset([train_dataset_normal, train_dataset_trim, train_dataset_external])
eval_dataset = CombineAICDataset([eval_dataset_normal, eval_dataset_trim, eval_dataset_external])

# Create collator
data_collator = VidLangCollator(tokenizer=tokenizer, mlm=False)

trainable_params: 150219008
total_params: 1568489728


In [4]:
lr = 5e-5
train_batch = 2

arguments = TrainingArguments(
    output_dir='./llamaer_checkpoints',
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=train_batch,
    per_device_eval_batch_size=train_batch,
    gradient_accumulation_steps=2,
    num_train_epochs=4,
#     max_steps=2,
    learning_rate=lr,
    warmup_steps=100,
    save_strategy='steps',
    save_steps=500,
    save_total_limit=3,
    eval_steps=100,
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=arguments,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
#     optimizers=torch.optim.Adam(model.parameters(), lr=lr),
)

trainer.train()

print(model)

wandb: Currently logged in as: npvinh20 (medical-llm-thesis). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240306_173948-q6c5e73p
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run electric-universe-58
wandb: ⭐️ View project at https://wandb.ai/medical-llm-thesis/huggingface
wandb: 🚀 View run at https://wandb.ai/medical-llm-thesis/huggingface/runs/q6c5e73p
/kaggle/working/phiver/dataset/collator.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pad_raw = (torch.tensor(max_shape) - torch.tensor(tensor_shape)).tolist()


Step,Training Loss
500,92.832500
1000,142.547800
1500,12.304700
2000,204.559700
2500,10.255800
3000,28.246500
3500,12.780800
4000,5.397900
4500,17.702600
5000,19.701600


/opt/conda/lib/python3.10/site-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in  - will assume that the vocabulary was not modified.
  warnings.warn(
/kaggle/working/phiver/dataset/collator.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pad_raw = (torch.tensor(max_shape) - torch.tensor(tensor_shape)).tolist()
/opt/conda/lib/python3.10/site-packages/transformers/integrations/peft.py:391: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a co

VideoPhiForCausalLM(
  (embeding_condition): Embedding(10, 1024)
  (double_perceiver): DoublePerceiver(
    (videos_perceiver): PerceiverModel(
      (embeddings): PerceiverEmbeddings()
      (encoder): PerceiverEncoder(
        (cross_attention): PerceiverLayer(
          (attention): PerceiverAttention(
            (self): PerceiverSelfAttention(
              (layernorm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
              (layernorm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
              (query): Linear(in_features=1024, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): PerceiverSelfOutput(
              (dense): lora.Linear(
                (base_layer): Linear(in_features=256, out_features=1024, bias=True)
                (lora_dro

In [5]:
torch.save(model.base_model.state_dict(), './original')

In [6]:
load_model = VideoLlamaForCausalLM(config)
load_model.add_adapter(lora_config)
load_model.load_state_dict(torch.load('./original'))

<All keys matched successfully>

In [7]:
for p1, p2 in zip(model.parameters(), load_model.parameters()):
    if p1.to('cpu').data.ne(p2.data.to('cpu')).sum() > 0:
        print(False)
print(True)

True


In [ ]:
import torch
from dataset.dataset import AICDataset, CombineAICDataset
from dataset.collator import VidLangCollator
from transformers import AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from model.llamaer_configuration import *
from model.modeling_llamaer import *
from peft import LoraConfig, get_peft_model

if torch.cuda.is_available():
    # Set the device to CUDA (GPU)
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    # Set the device to CPU
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

# Change following paths to your own
TEST_CAPTIONS_DIR = '/kaggle/input/aic24-test/WTS_DATASET_PUBLIC_TEST/WTS_DATASET_PUBLIC_TEST/annotations/caption/test/public_challenge'
TEST_CAPTIONS_TRIM_DIR = '/kaggle/input/aic24-test/WTS_DATASET_PUBLIC_TEST/WTS_DATASET_PUBLIC_TEST/annotations/caption/test/public_challenge/normal_trimmed'
TEST_CAPTIONS_EXTERNAL_DIR = '/kaggle/input/aic24-test/WTS_DATASET_PUBLIC_TEST/WTS_DATASET_PUBLIC_TEST/external/BDD_PC_5K/annotations/caption/test/public_challenge'

TEST_VIDEOS_DIR = '/kaggle/input/aic24-videofeatures-test/matrix/train'
TEST_VIDEO_EXTERNAL_DIR = '/kaggle/input/aic24vidtestexternal/matrix/testexternal'

test_dataset = AICDataset(tokenizer=tokenizer, captions_path=TEST_CAPTIONS_DIR, videos_path=TEST_VIDEOS_DIR, is_test=True, device=device)
test_trim_dataset = AICDataset(tokenizer=tokenizer, captions_path=TEST_CAPTIONS_TRIM_DIR, videos_path=TEST_VIDEOS_DIR, is_test=True, device=device)
test_exte_dataset = AICDataset(tokenizer=tokenizer, captions_path=TEST_CAPTIONS_EXTERNAL_DIR, videos_path=TEST_VIDEO_EXTERNAL_DIR, is_test=True, dataset_type=1, device=device)
test_full_dataset = CombineAICDataset([test_dataset, test_trim_dataset, test_exte_dataset])

In [ ]:
import json
from tqdm import tqdm

if torch.cuda.is_available():
    # Set the device to CUDA (GPU)
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    # Set the device to CPU
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

model.to(device).eval()

file_path = "./predict.json"
json_objects = []

collator = VidLangCollator(tokenizer, mlm=False)
collator.inference = True

batch_size = 8
start = 0

for index in tqdm(range(start, len(test_full_dataset), batch_size)):
    
    final_index = index + batch_size
    if final_index >= len(test_full_dataset):
        final_index = len(test_full_dataset)
    
    datas = [test_full_dataset[i] for i in range(index, final_index)]
    __ids = [data.pop('id') for data in datas]
    phases = [data['phase'] for data in datas]
    times = [data['time'] for data in datas]
    
    inputs = collator(datas)
    
    output = model.generate(**inputs, max_length=200)
    
    texts = tokenizer.batch_decode(output)
    
    json_objects = json_objects + [{
        'id': __id,
        'phase': phase.tolist(),
        'text': text,
        'time': time.tolist()
    } for __id, phase, text, time in zip(__ids, phases, texts, times)]
    
    with open(file_path, 'w') as json_file:
        json.dump(json_objects, json_file, indent=4)    

In [ ]:
import json
from typing import Dict
from typing_extensions import Tuple

with open('./predict.json', 'r') as rf:
    datas = json.load(rf)

mapping_phases = {'prerecognition': 0, 'recognition': 1, 'judgment': 2, 'action': 3, 'avoidance': 4, "0": 0, "1": 1, "2": 2, "3": 3, "4": 4}

def phase2token(phase: int):
    mapping_tokens = {i * 2 + j: (i, j) for i in range(5) for j in range(2)}
    mapping_objects = {0: 'pedestrian', 1: 'vehicle'}

    (label, i) = mapping_tokens[phase]

    return label, mapping_objects[i]

def get_correct_json(ids: Tuple, values: Dict):
    json_path = ids[-1]
    phase = ids[1]

    with open(json_path, 'r') as rf:
        phases = json.load(rf)['event_phase']

    for i in phases:
        if phase == int(i['labels'][0]):
            i.update(values)

            return ids[0], i

reformed_data = {}
final_data = {}

for data in datas:
    caption_path = data.pop('id')
    phase, _object = phase2token(data.pop('phase')[0][0])
    text = data.pop('text')

    # Get ID caption belongs to
    with open(caption_path, 'r') as rf:                                                                                                                                 
        json_data = json.load(rf)
        
        if 'video_name' in json_data: # External
            _id = (json_data['video_name'].split('.')[0], phase, caption_path)
        elif 'vehicle_view' in json_data: # Vehicle view
            _id = (os.path.splitext(json_data['vehicle_view'])[0], phase, caption_path)
        else:
            _id = (caption_path.split('/')[-3], phase, caption_path)

    # Update, Create result
    if _id not in reformed_data:
        reformed_data[_id] = {
            f'caption_{_object}': text
        }
    else:
        reformed_data[_id].update({
                                 f'caption_{_object}': text
                             })

for k, v in reformed_data.items():
    _id, value = get_correct_json(ids=k, values=v)

    if _id in final_data:
        final_data[_id].append(value)
    else:
        final_data[_id] = [value]

with open('submit.json', 'w') as wf:
    json.dump(final_data, wf, indent=4)

In [ ]:
import json
from typing import Dict
from typing_extensions import Tuple

with open('./predict.json', 'r') as rf:
    datas = json.load(rf)

mapping_phases = {'prerecognition': 0, 'recognition': 1, 'judgment': 2, 'action': 3, 'avoidance': 4, "0": 0, "1": 1, "2": 2, "3": 3, "4": 4}

def phase2token(phase: int):
    mapping_tokens = {i * 2 + j: (i, j) for i in range(5) for j in range(2)}
    mapping_objects = {0: 'pedestrian', 1: 'vehicle'}

    (label, i) = mapping_tokens[phase]

    return label, mapping_objects[i]

def get_correct_json(ids: Tuple, values: Dict):
    json_path = ids[-1]
    phase = ids[1]

    with open(json_path, 'r') as rf:
        phases = json.load(rf)['event_phase']

    for i in phases:
        if phase == int(i['labels'][0]):
            i.update(values)

            return ids[0], i

reformed_data = {}
final_data = {}

for data in datas:
    caption_path = data.pop('id')
    phase, _object = phase2token(data.pop('phase')[0][0])
    text = data.pop('text')

    # Get ID caption belongs to
    with open(caption_path, 'r') as rf:                                                                                                                                 
        json_data = json.load(rf)
        
        if 'video_name' in json_data: # External
            _id = (json_data['video_name'].split('.')[0], phase, caption_path)
        elif 'vehicle_view' in json_data: # Vehicle view
            _id = (os.path.splitext(json_data['vehicle_view'])[0], phase, caption_path)
        else:
            _id = (caption_path.split('/')[-3], phase, caption_path)

    # Update, Create result
    if _id not in reformed_data:
        reformed_data[_id] = {
            f'caption_{_object}': text
        }
    else:
        reformed_data[_id].update({
                                 f'caption_{_object}': text
                             })

for k, v in reformed_data.items():
    _id, value = get_correct_json(ids=k, values=v)

    if _id in final_data:
        final_data[_id].append(value)
    else:
        final_data[_id] = [value]

with open('submit.json', 'w') as wf:
    json.dump(final_data, wf, indent=4)

import re
from typing import Optional, Tuple

def extract_main_text(raw_text: str) -> Optional[Tuple[float, float, str]]:
    """
    Extracts (start_time, end_time, main_text) from a structured caption input.

    Args:
        raw_text (str): Input caption line.

    Returns:
        Optional[Tuple[float, float, str]]: (start_time, end_time, main_text) or None if format invalid.
    """
    # Pattern to extract time and caption
    pattern = (
        r"time\s*\[\s*([\d.]+)\s*-\s*([\d.]+)\s*\]\s*:\s*event\s*:\s*(.*?)(?:<\|eos\|>|$)"
    )
    match = re.search(pattern, raw_text, re.IGNORECASE)
    
    if match:
        start_time = float(match.group(1))
        end_time = float(match.group(2))
        main_text = match.group(3)

        # Clean up main text: remove leading/trailing spaces and !
        main_text = main_text.strip("! \n\t")

        # Collapse repeated ! if needed
        main_text = re.sub(r"^!+|!+$", "", main_text).strip()

        return (start_time, end_time, main_text)
    
    return None

def clean_captions(data: dict) -> dict:
    """
    Given a dictionary containing video entries with vehicle and pedestrian captions,
    return a dictionary with cleaned captions.
    """
    cleaned_data = {}
    for video_id, entries in data.items():
        cleaned_entries = []
        for entry in entries:
            cleaned_entry = entry.copy()
            if 'caption_vehicle' in entry:
                cleaned_entry['start_vehicle'],cleaned_entry['end_vehicle'],cleaned_entry['caption_vehicle'] = extract_main_text(entry['caption_vehicle'])
            if 'caption_pedestrian' in entry:
                cleaned_entry['start_pedestrian'],cleaned_entry['end_pedestrian'],cleaned_entry['caption_pedestrian'] = extract_main_text(entry['caption_pedestrian'])
            cleaned_entries.append(cleaned_entry)
        cleaned_data[video_id] = cleaned_entries
    return cleaned_data

with open('submit.json') as rf:
    data = json.load(rf)

with open('submit_clean.json', 'w') as wf:
    json.dump(clean_captions(data), wf, indent=4)